In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Tocantins - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_TO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Tocantins - Desemprego,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - Produção de Cimento (t),Tocantins - IDH,Tocantins - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.243830,1.593908e+07,1.457380e+06,10.276585,2.572692e+07,12.733000,0.700107,17.555
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.237931,1.596108e+07,1.459029e+06,10.281590,2.576177e+07,12.733000,0.700487,14.976
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.232033,1.598308e+07,1.460678e+06,10.286595,2.579662e+07,12.733000,0.700866,13.334
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.226135,1.600508e+07,1.462328e+06,10.291600,2.583147e+07,12.733000,0.701246,15.173
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.220236,1.602709e+07,1.463977e+06,10.296605,2.586632e+07,12.733000,0.701626,18.728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.097178,NaN,54.154
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.077411,NaN,55.664
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.563807,NaN,52.320
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.682099,NaN,43.036


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Tocantins - Desemprego,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - Produção de Cimento (t),Tocantins - IDH
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.738973,-1.644217,0.633315,-2.414213,0.326404,-1.246730,-2.226834
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.745082,-1.628369,0.646316,-2.377931,0.345573,-1.246730,-2.188140
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.751192,-1.612520,0.659318,-2.341648,0.364742,-1.246730,-2.149447
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.757301,-1.596671,0.672320,-2.305365,0.383911,-1.246730,-2.110754
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.763410,-1.580822,0.685321,-2.269082,0.403080,-1.246730,-2.072060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.745989,1.219532,-0.763386,1.120755,-0.578577,1.289207,1.307164
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.733597,1.213944,-0.756340,1.118160,-0.567586,1.310568,1.296751
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.721205,1.208357,-0.749293,1.115565,-0.556595,1.345357,1.286338
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.708813,1.202769,-0.742246,1.112970,-0.545604,1.340245,1.275925


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      19.115
1      22.973
2      30.126
3      29.497
4      29.528
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Tocantins - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Tocantins - Desemprego,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - Produção de Cimento (t),Tocantins - IDH
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.738973,-1.644217,0.633315,-2.414213,0.326404,-1.246730,-2.226834
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.745082,-1.628369,0.646316,-2.377931,0.345573,-1.246730,-2.188140
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.751192,-1.612520,0.659318,-2.341648,0.364742,-1.246730,-2.149447
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.757301,-1.596671,0.672320,-2.305365,0.383911,-1.246730,-2.110754
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.763410,-1.580822,0.685321,-2.269082,0.403080,-1.246730,-2.072060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.443839,1.208594,-1.016210,1.001982,-0.899654,0.819052,1.403526
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.400830,1.213994,-1.006113,1.012925,-0.888702,0.823544,1.404858
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.357822,1.219395,-0.996016,1.023868,-0.877750,0.828158,1.406191
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.314814,1.224795,-0.985919,1.034811,-0.866797,0.835894,1.407523


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      19.115
1      22.973
2      30.126
3      29.497
4      29.528
        ...  
157    33.787
158    41.594
159    36.019
160    45.143
161    41.704
Name: Tocantins - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 14)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Tocantins - Desemprego,Tocantins - PIB - Estadual,Tocantins - PIB - Construção Civil,Tocantins - PIB - Per Capita,Tocantins - PIB - Preços de Mercado,Tocantins - Produção de Cimento (t),Tocantins - IDH
126,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,1.088278,0.772402,-1.145973,0.569439,-1.230338,0.880512,0.413591
127,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,1.120223,0.791690,-1.142277,0.586987,-1.219642,0.876072,0.418361
128,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,1.152168,0.810977,-1.138581,0.604534,-1.208947,0.915172,0.423132
129,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,1.184114,0.830265,-1.134885,0.622082,-1.198251,0.928517,0.427902
130,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,1.216059,0.849553,-1.131189,0.639630,-1.187556,0.926782,0.432672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.745989,1.219532,-0.763386,1.120755,-0.578577,1.289207,1.307164
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.733597,1.213944,-0.756340,1.118160,-0.567586,1.310568,1.296751
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.721205,1.208357,-0.749293,1.115565,-0.556595,1.345357,1.286338
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.708813,1.202769,-0.742246,1.112970,-0.545604,1.340245,1.275925


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 14)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3396024063, 3356502310, 66687958, 3425957406, 3093936016, 3007331220, 2479639783, 4068593882, 3382453581, 4247728713, 4206961639, 772331694, 4164619101, 1794083268, 2025432263, 1093472367, 2161171174, 3962795557, 79924732, 1987005629, 1661347759, 508489345, 133140738, 1963959920, 73395237, 2613713928, 1648055650, 720690221, 1438349413, 4193458938, 2057728990, 3372340053, 2844296341, 280733174, 794962131, 3584158966, 2227741619, 617131700, 3177550705, 1314443564, 1970635701, 63446628, 3955870562, 3174371010, 433009310, 2357752301, 3998676436, 388385700, 848187785, 2451617468]


Step: 0 ___________________________________________
val_loss: 9.18000602722168
winner_seed: 3396024063


Step: 1 ___________________________________________
val_loss: 10.284198760986328


Step: 2 ___________________________________________
val_loss: 10.474555969238281


Step: 3 ___________________________________________
val_loss: 8.033976554870605
winner_seed: 3425957406


Step: 4 ______________________________

In [35]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 133ms/step - loss: 1541.8782 - val_loss: 58.5674
Epoch 2/10000
4/4 [==============================] - 0s 11ms/step - loss: 55.0375 - val_loss: 40.0912
Epoch 3/10000
4/4 [==============================] - 0s 11ms/step - loss: 45.0604 - val_loss: 56.1965
Epoch 4/10000
4/4 [==============================] - 0s 11ms/step - loss: 48.1574 - val_loss: 36.4243
Epoch 5/10000
4/4 [==============================] - 0s 11ms/step - loss: 40.5622 - val_loss: 85.3874
Epoch 6/10000
4/4 [==============================] - 0s 11ms/step - loss: 78.9591 - val_loss: 46.2543
Epoch 7/10000
4/4 [==============================] - 0s 12ms/step - loss: 40.3858 - val_loss: 31.7986
Epoch 8/10000
4/4 [==============================] - 0s 12ms/step - loss: 37.8734 - val_loss: 47.4215
Epoch 9/10000
4/4 [==============================] - 0s 12ms/step - loss: 41.8549 - val_loss: 29.4631
Epoch 10/10000
4/4 [==============================] - 0s 12ms/step - loss: 34.3

4/4 [==============================] - 0s 11ms/step - loss: 27.7297 - val_loss: 30.2684
Epoch 81/10000
4/4 [==============================] - 0s 12ms/step - loss: 27.2702 - val_loss: 29.8384
Epoch 82/10000
4/4 [==============================] - 0s 11ms/step - loss: 26.0479 - val_loss: 27.5023
Epoch 83/10000
4/4 [==============================] - 0s 11ms/step - loss: 24.3105 - val_loss: 45.9856
Epoch 84/10000
4/4 [==============================] - 0s 10ms/step - loss: 32.7970 - val_loss: 25.9963
Epoch 85/10000
4/4 [==============================] - 0s 11ms/step - loss: 24.5675 - val_loss: 46.6933
Epoch 86/10000
4/4 [==============================] - 0s 11ms/step - loss: 27.3984 - val_loss: 26.3405
Epoch 87/10000
4/4 [==============================] - 0s 10ms/step - loss: 25.3299 - val_loss: 27.2144
Epoch 88/10000
4/4 [==============================] - 0s 11ms/step - loss: 25.1471 - val_loss: 28.4418
Epoch 89/10000
4/4 [==============================] - 0s 11ms/step - loss: 25.9692 - val

4/4 [==============================] - 0s 11ms/step - loss: 17.8585 - val_loss: 24.5390
Epoch 160/10000
4/4 [==============================] - 0s 11ms/step - loss: 17.6624 - val_loss: 31.2652
Epoch 161/10000
4/4 [==============================] - 0s 11ms/step - loss: 28.4790 - val_loss: 17.8573
Epoch 162/10000
4/4 [==============================] - 0s 10ms/step - loss: 14.8483 - val_loss: 15.3665
Epoch 163/10000
4/4 [==============================] - 0s 10ms/step - loss: 17.2427 - val_loss: 14.9417
Epoch 164/10000
4/4 [==============================] - 0s 10ms/step - loss: 17.3657 - val_loss: 16.5984
Epoch 165/10000
4/4 [==============================] - 0s 11ms/step - loss: 15.7622 - val_loss: 19.2994
Epoch 166/10000
4/4 [==============================] - 0s 10ms/step - loss: 16.7682 - val_loss: 23.8996
Epoch 167/10000
4/4 [==============================] - 0s 10ms/step - loss: 18.7934 - val_loss: 15.2870
Epoch 168/10000
4/4 [==============================] - 0s 10ms/step - loss: 15.0

Epoch 238/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.8612 - val_loss: 10.7386
Epoch 239/10000
4/4 [==============================] - 0s 11ms/step - loss: 11.4281 - val_loss: 15.7699
Epoch 240/10000
4/4 [==============================] - 0s 11ms/step - loss: 9.8546 - val_loss: 17.6155
Epoch 241/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.3686 - val_loss: 12.7035
Epoch 242/10000
4/4 [==============================] - 0s 10ms/step - loss: 9.2553 - val_loss: 12.1327
Epoch 243/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.5180 - val_loss: 11.1871
Epoch 244/10000
4/4 [==============================] - 0s 10ms/step - loss: 12.1451 - val_loss: 24.2388
Epoch 245/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.5924 - val_loss: 12.6558
Epoch 246/10000
4/4 [==============================] - 0s 11ms/step - loss: 10.5670 - val_loss: 12.5142
Epoch 247/10000
4/4 [==============================] - 0s 10ms/step -

4/4 [==============================] - 0s 11ms/step - loss: 8.6540 - val_loss: 12.4115
Epoch 318/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.8268 - val_loss: 14.5207
Epoch 319/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.9020 - val_loss: 18.5924
Epoch 320/10000
4/4 [==============================] - 0s 10ms/step - loss: 11.3608 - val_loss: 15.0367
Epoch 321/10000
4/4 [==============================] - 0s 10ms/step - loss: 9.1656 - val_loss: 14.1457
Epoch 322/10000
4/4 [==============================] - 0s 10ms/step - loss: 11.5649 - val_loss: 9.7965
Epoch 323/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.8716 - val_loss: 11.9438
Epoch 324/10000
4/4 [==============================] - 0s 10ms/step - loss: 9.2862 - val_loss: 9.9830
Epoch 325/10000
4/4 [==============================] - 0s 10ms/step - loss: 9.2707 - val_loss: 8.9962
Epoch 326/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.9868 - val_l

...

4/4 [==============================] - 0s 8ms/step - loss: 6.3685 - val_loss: 13.8487
Epoch 715/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.5725 - val_loss: 15.6652
Epoch 716/10000
4/4 [==============================] - 0s 9ms/step - loss: 6.6349 - val_loss: 14.1019
Epoch 717/10000
4/4 [==============================] - 0s 9ms/step - loss: 5.8491 - val_loss: 17.5474
Epoch 718/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.3370 - val_loss: 12.0525
Epoch 719/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.0550 - val_loss: 12.2496
Epoch 720/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.1838 - val_loss: 12.5075
Epoch 721/10000
4/4 [==============================] - 0s 9ms/step - loss: 6.2273 - val_loss: 14.4240
Epoch 722/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.8124 - val_loss: 16.5411
Epoch 723/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.5156 - val_loss

Epoch 794/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.2111 - val_loss: 11.8405
Epoch 795/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.3744 - val_loss: 12.4400
Epoch 796/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.1166 - val_loss: 12.8106
Epoch 797/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.9708 - val_loss: 18.3642
Epoch 798/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.5558 - val_loss: 9.1001
Epoch 799/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.6876 - val_loss: 16.6322
Epoch 800/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.3935 - val_loss: 11.6996
Epoch 801/10000
4/4 [==============================] - 0s 12ms/step - loss: 6.1037 - val_loss: 14.2857
Epoch 802/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.5755 - val_loss: 11.1151
Epoch 803/10000
4/4 [==============================] - 0s 9ms/step - loss:

4/4 [==============================] - 0s 11ms/step - loss: 5.5081 - val_loss: 12.4867
Epoch 874/10000
4/4 [==============================] - 0s 11ms/step - loss: 4.3911 - val_loss: 12.4528
Epoch 875/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.2313 - val_loss: 15.4104
Epoch 876/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.5727 - val_loss: 17.2958
Epoch 877/10000
4/4 [==============================] - 0s 11ms/step - loss: 4.8931 - val_loss: 19.0689
Epoch 878/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.4758 - val_loss: 15.4994
Epoch 879/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.0835 - val_loss: 15.0273
Epoch 880/10000
4/4 [==============================] - 0s 9ms/step - loss: 5.1892 - val_loss: 13.6123
Epoch 881/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.7651 - val_loss: 15.0730
Epoch 882/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.4639 - val_l

Epoch 953/10000
4/4 [==============================] - 0s 9ms/step - loss: 4.7145 - val_loss: 13.1757
Epoch 954/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.6815 - val_loss: 14.1023
Epoch 955/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.6441 - val_loss: 12.0614
Epoch 956/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.5148 - val_loss: 17.6907
Epoch 957/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.2063 - val_loss: 15.3550
Epoch 958/10000
4/4 [==============================] - 0s 9ms/step - loss: 6.0650 - val_loss: 13.2003
Epoch 959/10000
4/4 [==============================] - 0s 11ms/step - loss: 4.9108 - val_loss: 14.3032
Epoch 960/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.0958 - val_loss: 16.6149
Epoch 961/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.0679 - val_loss: 14.2435
Epoch 962/10000
4/4 [==============================] - 0s 10ms/step - loss:

4/4 [==============================] - 0s 10ms/step - loss: 4.4406 - val_loss: 17.7189
Epoch 1033/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.9192 - val_loss: 14.8541
Epoch 1034/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.1102 - val_loss: 11.9778
Epoch 1035/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.2056 - val_loss: 11.9889
Epoch 1036/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.1207 - val_loss: 14.8026
Epoch 1037/10000
4/4 [==============================] - 0s 13ms/step - loss: 4.7888 - val_loss: 13.0932
Epoch 1037: early stopping


In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 16ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,40.28421,40.37804,40.468311,40.532639,40.573151,40.566189,40.59388,40.751869,40.972759,41.156002,41.295074,41.478939,41.500256,41.564415,41.81469,42.044613,41.988037,41.769043,41.769367,42.186474,42.858295,43.352283,43.718281,43.862797,43.846409,43.724384,43.556164,43.670437,44.148746,44.557957
Target,45.71,45.564,43.754,48.554,40.896,35.255,34.018,22.422,29.555,28.919,25.589,40.237,37.012,38.793,38.281,39.711,33.695,28.206,31.615,30.275,29.568,26.94,35.91,33.954,37.983,40.046,32.647,39.065,36.18,28.749
Error,5.425789,5.185959,3.28569,8.021362,0.322849,5.311188,6.575878,18.329868,11.417759,12.237001,15.706074,1.24194,4.488255,2.771416,3.533691,2.333614,8.293037,13.563044,10.154367,11.911474,13.290295,16.412283,7.808281,9.908798,5.863407,3.678383,10.909164,4.605438,7.968746,15.808956


In [38]:
display(mae)
display(mape)

8.212133

0.2639352

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model,
                                             reshaped_test, 
                                             reshaped_test_target,
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[489.05106]] - Target[440.473]| =  Error: [[48.578064]]; MAPE:[[0.11028614]]
1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[508.42856]] - Target[403.96]| =  Error: [[104.46857]]; MAPE:[[0.25861117]]
1/1 [==============================] - 0s 15ms/step
Ano-5: |Prediction[[263.5041]] - Target[214.67]| =  Error: [[48.83409]]; MAPE:[[0.22748446]]


[array([[48.578064]], dtype=float32),
 array([[104.46857]], dtype=float32),
 array([[48.83409]], dtype=float32)]

67.29357

0.19879393